In [1]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
 

Using TensorFlow backend.
C:\Users\admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

# Model 1: One-Word-In, One-Word-Out Sequences

In [2]:
# generate a sequence from the model
def generate_seq(model, tokenizer, seed_text, n_words):
	in_text, result = seed_text, seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		encoded = array(encoded)
		# predict a word in the vocabulary
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text, result = out_word, result + ' ' + out_word
	return result

In [3]:
# source text
data = """ Jack and Jill went up the hill\n
		To fetch a pail of water\n
		Jack fell down and broke his crown\n
		And Jill came tumbling after\n """

In [4]:
# integer encode text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]

In [5]:
encoded

[2,
 1,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 2,
 14,
 15,
 1,
 16,
 17,
 18,
 1,
 3,
 19,
 20,
 21]

In [6]:
# determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 22


In [7]:
# create word -> word sequences
sequences = list()
for i in range(1, len(encoded)):
	sequence = encoded[i-1:i+1]
	sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 24


In [8]:
sequences

[[2, 1],
 [1, 3],
 [3, 4],
 [4, 5],
 [5, 6],
 [6, 7],
 [7, 8],
 [8, 9],
 [9, 10],
 [10, 11],
 [11, 12],
 [12, 13],
 [13, 2],
 [2, 14],
 [14, 15],
 [15, 1],
 [1, 16],
 [16, 17],
 [17, 18],
 [18, 1],
 [1, 3],
 [3, 19],
 [19, 20],
 [20, 21]]

In [9]:
# split into X and y elements
sequences = array(sequences)
X, y = sequences[:,0],sequences[:,1]

In [10]:
X

array([ 2,  1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  2, 14, 15,  1,
       16, 17, 18,  1,  3, 19, 20])

In [11]:
y

array([ 1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  2, 14, 15,  1, 16,
       17, 18,  1,  3, 19, 20, 21])

In [12]:
# one hot encode outputs
y = to_categorical(y, num_classes=vocab_size)

In [13]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1, 10)             220       
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_1 (Dense)              (None, 22)                1122      
Total params: 13,542
Trainable params: 13,542
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
# fit network
model.fit(X, y, epochs=500, verbose=2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/500
 - 2s - loss: 3.0918 - acc: 0.0417
Epoch 2/500
 - 0s - loss: 3.0910 - acc: 0.0417
Epoch 3/500
 - 0s - loss: 3.0902 - acc: 0.0833
Epoch 4/500
 - 0s - loss: 3.0894 - acc: 0.1250
Epoch 5/500
 - 0s - loss: 3.0885 - acc: 0.1667
Epoch 6/500
 - 0s - loss: 3.0877 - acc: 0.2083
Epoch 7/500
 - 0s - loss: 3.0869 - acc: 0.2083
Epoch 8/500
 - 0s - loss: 3.0861 - acc: 0.2083
Epoch 9/500
 - 0s - loss: 3.0853 - acc: 0.2083
Epoch 10/500
 - 0s - loss: 3.0844 - acc: 0.2083
Epoch 11/500
 - 0s - loss: 3.0836 - acc: 0.2083
Epoch 12/500
 - 0s - loss: 3.0828 - acc: 0.2083
Epoch 13/500
 - 0s - loss: 3.0819 - acc: 0.2083
Epoch 14/500
 - 0s - loss: 3.0811 - acc: 0.2083
Epoch 15/500
 - 0s - loss: 3.0802 - acc: 0.2083
Epoch 16/500
 - 0s - loss: 3.0793 - acc: 0.2083
Epoch 17/500
 - 0s - loss: 3.0784 - acc: 0.2083
Epoch 18/500
 - 0s - loss: 3.0775 - acc: 0.2083
Epoch 19/500
 - 0s - loss: 3.0766 - acc: 0.2083
Ep

Epoch 160/500
 - 0s - loss: 2.0799 - acc: 0.5417
Epoch 161/500
 - 0s - loss: 2.0643 - acc: 0.5417
Epoch 162/500
 - 0s - loss: 2.0487 - acc: 0.5417
Epoch 163/500
 - 0s - loss: 2.0331 - acc: 0.6250
Epoch 164/500
 - 0s - loss: 2.0175 - acc: 0.6250
Epoch 165/500
 - 0s - loss: 2.0018 - acc: 0.6250
Epoch 166/500
 - 0s - loss: 1.9861 - acc: 0.6250
Epoch 167/500
 - 0s - loss: 1.9705 - acc: 0.6250
Epoch 168/500
 - 0s - loss: 1.9548 - acc: 0.6250
Epoch 169/500
 - 0s - loss: 1.9391 - acc: 0.6250
Epoch 170/500
 - 0s - loss: 1.9234 - acc: 0.6250
Epoch 171/500
 - 0s - loss: 1.9078 - acc: 0.6250
Epoch 172/500
 - 0s - loss: 1.8921 - acc: 0.6250
Epoch 173/500
 - 0s - loss: 1.8764 - acc: 0.6250
Epoch 174/500
 - 0s - loss: 1.8608 - acc: 0.6250
Epoch 175/500
 - 0s - loss: 1.8451 - acc: 0.6250
Epoch 176/500
 - 0s - loss: 1.8295 - acc: 0.6667
Epoch 177/500
 - 0s - loss: 1.8139 - acc: 0.6667
Epoch 178/500
 - 0s - loss: 1.7983 - acc: 0.6667
Epoch 179/500
 - 0s - loss: 1.7827 - acc: 0.6667
Epoch 180/500
 - 0s 

Epoch 328/500
 - 0s - loss: 0.3805 - acc: 0.8750
Epoch 329/500
 - 0s - loss: 0.3778 - acc: 0.8750
Epoch 330/500
 - 0s - loss: 0.3751 - acc: 0.8750
Epoch 331/500
 - 0s - loss: 0.3725 - acc: 0.8750
Epoch 332/500
 - 0s - loss: 0.3700 - acc: 0.8750
Epoch 333/500
 - 0s - loss: 0.3675 - acc: 0.8750
Epoch 334/500
 - 0s - loss: 0.3650 - acc: 0.8750
Epoch 335/500
 - 0s - loss: 0.3626 - acc: 0.8750
Epoch 336/500
 - 0s - loss: 0.3602 - acc: 0.8750
Epoch 337/500
 - 0s - loss: 0.3578 - acc: 0.8750
Epoch 338/500
 - 0s - loss: 0.3556 - acc: 0.8750
Epoch 339/500
 - 0s - loss: 0.3533 - acc: 0.8750
Epoch 340/500
 - 0s - loss: 0.3511 - acc: 0.8750
Epoch 341/500
 - 0s - loss: 0.3489 - acc: 0.8750
Epoch 342/500
 - 0s - loss: 0.3468 - acc: 0.8750
Epoch 343/500
 - 0s - loss: 0.3447 - acc: 0.8750
Epoch 344/500
 - 0s - loss: 0.3426 - acc: 0.8750
Epoch 345/500
 - 0s - loss: 0.3406 - acc: 0.8750
Epoch 346/500
 - 0s - loss: 0.3386 - acc: 0.8750
Epoch 347/500
 - 0s - loss: 0.3367 - acc: 0.8750
Epoch 348/500
 - 0s 

Epoch 496/500
 - 0s - loss: 0.2288 - acc: 0.8750
Epoch 497/500
 - 0s - loss: 0.2286 - acc: 0.8750
Epoch 498/500
 - 0s - loss: 0.2283 - acc: 0.8750
Epoch 499/500
 - 0s - loss: 0.2281 - acc: 0.8750
Epoch 500/500
 - 0s - loss: 0.2279 - acc: 0.8750


In [16]:
# evaluate
print(generate_seq(model, tokenizer, 'Jack', 6))


Jack and jill went up the hill


# Model 2: Line-by-Line Sequence

In [17]:
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [18]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# pre-pad sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
	return in_text

In [19]:
# source text
data = """ Jack and Jill went up the hill\n
		To fetch a pail of water\n
		Jack fell down and broke his crown\n
		And Jill came tumbling after\n """

In [20]:
# prepare the tokenizer on the source text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

In [21]:
# determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 22


In [22]:
# create line-based sequences
sequences = list()
for line in data.split('\n'):
	encoded = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(encoded)):
		sequence = encoded[:i+1]
		sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 21


In [23]:
sequences

[[2, 1],
 [2, 1, 3],
 [2, 1, 3, 4],
 [2, 1, 3, 4, 5],
 [2, 1, 3, 4, 5, 6],
 [2, 1, 3, 4, 5, 6, 7],
 [8, 9],
 [8, 9, 10],
 [8, 9, 10, 11],
 [8, 9, 10, 11, 12],
 [8, 9, 10, 11, 12, 13],
 [2, 14],
 [2, 14, 15],
 [2, 14, 15, 1],
 [2, 14, 15, 1, 16],
 [2, 14, 15, 1, 16, 17],
 [2, 14, 15, 1, 16, 17, 18],
 [1, 3],
 [1, 3, 19],
 [1, 3, 19, 20],
 [1, 3, 19, 20, 21]]

In [24]:
# pad input sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)

Max Sequence Length: 7


In [25]:
sequences

array([[ 0,  0,  0,  0,  0,  2,  1],
       [ 0,  0,  0,  0,  2,  1,  3],
       [ 0,  0,  0,  2,  1,  3,  4],
       [ 0,  0,  2,  1,  3,  4,  5],
       [ 0,  2,  1,  3,  4,  5,  6],
       [ 2,  1,  3,  4,  5,  6,  7],
       [ 0,  0,  0,  0,  0,  8,  9],
       [ 0,  0,  0,  0,  8,  9, 10],
       [ 0,  0,  0,  8,  9, 10, 11],
       [ 0,  0,  8,  9, 10, 11, 12],
       [ 0,  8,  9, 10, 11, 12, 13],
       [ 0,  0,  0,  0,  0,  2, 14],
       [ 0,  0,  0,  0,  2, 14, 15],
       [ 0,  0,  0,  2, 14, 15,  1],
       [ 0,  0,  2, 14, 15,  1, 16],
       [ 0,  2, 14, 15,  1, 16, 17],
       [ 2, 14, 15,  1, 16, 17, 18],
       [ 0,  0,  0,  0,  0,  1,  3],
       [ 0,  0,  0,  0,  1,  3, 19],
       [ 0,  0,  0,  1,  3, 19, 20],
       [ 0,  0,  1,  3, 19, 20, 21]])

In [26]:
# split into input and output elements
sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)

In [27]:
X

array([[ 0,  0,  0,  0,  0,  2],
       [ 0,  0,  0,  0,  2,  1],
       [ 0,  0,  0,  2,  1,  3],
       [ 0,  0,  2,  1,  3,  4],
       [ 0,  2,  1,  3,  4,  5],
       [ 2,  1,  3,  4,  5,  6],
       [ 0,  0,  0,  0,  0,  8],
       [ 0,  0,  0,  0,  8,  9],
       [ 0,  0,  0,  8,  9, 10],
       [ 0,  0,  8,  9, 10, 11],
       [ 0,  8,  9, 10, 11, 12],
       [ 0,  0,  0,  0,  0,  2],
       [ 0,  0,  0,  0,  2, 14],
       [ 0,  0,  0,  2, 14, 15],
       [ 0,  0,  2, 14, 15,  1],
       [ 0,  2, 14, 15,  1, 16],
       [ 2, 14, 15,  1, 16, 17],
       [ 0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  1,  3],
       [ 0,  0,  0,  1,  3, 19],
       [ 0,  0,  1,  3, 19, 20]])

In [28]:
y

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [29]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_length-1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 6, 10)             220       
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_2 (Dense)              (None, 22)                1122      
Total params: 13,542
Trainable params: 13,542
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
# fit network
model.fit(X, y, epochs=500, verbose=2)

Epoch 1/500
 - 2s - loss: 3.0912 - acc: 0.0476
Epoch 2/500
 - 0s - loss: 3.0895 - acc: 0.0952
Epoch 3/500
 - 0s - loss: 3.0879 - acc: 0.1429
Epoch 4/500
 - 0s - loss: 3.0863 - acc: 0.1905
Epoch 5/500
 - 0s - loss: 3.0846 - acc: 0.1905
Epoch 6/500
 - 0s - loss: 3.0829 - acc: 0.2381
Epoch 7/500
 - 0s - loss: 3.0812 - acc: 0.1905
Epoch 8/500
 - 0s - loss: 3.0794 - acc: 0.1429
Epoch 9/500
 - 0s - loss: 3.0776 - acc: 0.0952
Epoch 10/500
 - 0s - loss: 3.0757 - acc: 0.0952
Epoch 11/500
 - 0s - loss: 3.0737 - acc: 0.0952
Epoch 12/500
 - 0s - loss: 3.0716 - acc: 0.0952
Epoch 13/500
 - 0s - loss: 3.0695 - acc: 0.0952
Epoch 14/500
 - 0s - loss: 3.0672 - acc: 0.0952
Epoch 15/500
 - 0s - loss: 3.0648 - acc: 0.0952
Epoch 16/500
 - 0s - loss: 3.0623 - acc: 0.0952
Epoch 17/500
 - 0s - loss: 3.0596 - acc: 0.0952
Epoch 18/500
 - 0s - loss: 3.0568 - acc: 0.0952
Epoch 19/500
 - 0s - loss: 3.0538 - acc: 0.0952
Epoch 20/500
 - 0s - loss: 3.0505 - acc: 0.0952
Epoch 21/500
 - 0s - loss: 3.0471 - acc: 0.0952
E

Epoch 171/500
 - 0s - loss: 0.6542 - acc: 0.8571
Epoch 172/500
 - 0s - loss: 0.6489 - acc: 0.8571
Epoch 173/500
 - 0s - loss: 0.6439 - acc: 0.8571
Epoch 174/500
 - 0s - loss: 0.6391 - acc: 0.8571
Epoch 175/500
 - 0s - loss: 0.6344 - acc: 0.8571
Epoch 176/500
 - 0s - loss: 0.6296 - acc: 0.8571
Epoch 177/500
 - 0s - loss: 0.6248 - acc: 0.8571
Epoch 178/500
 - 0s - loss: 0.6202 - acc: 0.8571
Epoch 179/500
 - 0s - loss: 0.6157 - acc: 0.8571
Epoch 180/500
 - 0s - loss: 0.6112 - acc: 0.8571
Epoch 181/500
 - 0s - loss: 0.6068 - acc: 0.8571
Epoch 182/500
 - 0s - loss: 0.6024 - acc: 0.8571
Epoch 183/500
 - 0s - loss: 0.5982 - acc: 0.8571
Epoch 184/500
 - 0s - loss: 0.5940 - acc: 0.8571
Epoch 185/500
 - 0s - loss: 0.5898 - acc: 0.8571
Epoch 186/500
 - 0s - loss: 0.5858 - acc: 0.8571
Epoch 187/500
 - 0s - loss: 0.5817 - acc: 0.8571
Epoch 188/500
 - 0s - loss: 0.5778 - acc: 0.8571
Epoch 189/500
 - 0s - loss: 0.5738 - acc: 0.8571
Epoch 190/500
 - 0s - loss: 0.5700 - acc: 0.8571
Epoch 191/500
 - 0s 

 - 0s - loss: 0.2229 - acc: 0.9524
Epoch 339/500
 - 0s - loss: 0.2216 - acc: 0.9524
Epoch 340/500
 - 0s - loss: 0.2201 - acc: 0.9524
Epoch 341/500
 - 0s - loss: 0.2188 - acc: 0.9524
Epoch 342/500
 - 0s - loss: 0.2174 - acc: 0.9524
Epoch 343/500
 - 0s - loss: 0.2160 - acc: 0.9524
Epoch 344/500
 - 0s - loss: 0.2147 - acc: 0.9524
Epoch 345/500
 - 0s - loss: 0.2133 - acc: 0.9524
Epoch 346/500
 - 0s - loss: 0.2120 - acc: 0.9524
Epoch 347/500
 - 0s - loss: 0.2109 - acc: 0.9524
Epoch 348/500
 - 0s - loss: 0.2095 - acc: 0.9524
Epoch 349/500
 - 0s - loss: 0.2082 - acc: 0.9524
Epoch 350/500
 - 0s - loss: 0.2070 - acc: 0.9524
Epoch 351/500
 - 0s - loss: 0.2057 - acc: 0.9524
Epoch 352/500
 - 0s - loss: 0.2046 - acc: 0.9524
Epoch 353/500
 - 0s - loss: 0.2032 - acc: 0.9524
Epoch 354/500
 - 0s - loss: 0.2021 - acc: 0.9524
Epoch 355/500
 - 0s - loss: 0.2009 - acc: 0.9524
Epoch 356/500
 - 0s - loss: 0.1997 - acc: 0.9524
Epoch 357/500
 - 0s - loss: 0.1985 - acc: 0.9524
Epoch 358/500
 - 0s - loss: 0.1973

In [32]:
# evaluate model
print(generate_seq(model, tokenizer, max_length-1, 'Jack', 4))
print(generate_seq(model, tokenizer, max_length-1, 'Jill', 4))

Jack fell down and broke
Jill jill came tumbling after


# Model 3: Two-Words-In, One-Word-Out Sequence

In [33]:
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [34]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# pre-pad sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
	return in_text
 

In [35]:
# source text
data = """ Jack and Jill went up the hill\n
		To fetch a pail of water\n
		Jack fell down and broke his crown\n
		And Jill came tumbling after\n """

In [36]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]

In [37]:
encoded

[2,
 1,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 2,
 14,
 15,
 1,
 16,
 17,
 18,
 1,
 3,
 19,
 20,
 21]

In [38]:
# retrieve vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 22


In [39]:
# encode 2 words -> 1 word
sequences = list()
for i in range(2, len(encoded)):
	sequence = encoded[i-2:i+1]
	sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 23


In [40]:
sequences

[[2, 1, 3],
 [1, 3, 4],
 [3, 4, 5],
 [4, 5, 6],
 [5, 6, 7],
 [6, 7, 8],
 [7, 8, 9],
 [8, 9, 10],
 [9, 10, 11],
 [10, 11, 12],
 [11, 12, 13],
 [12, 13, 2],
 [13, 2, 14],
 [2, 14, 15],
 [14, 15, 1],
 [15, 1, 16],
 [1, 16, 17],
 [16, 17, 18],
 [17, 18, 1],
 [18, 1, 3],
 [1, 3, 19],
 [3, 19, 20],
 [19, 20, 21]]

In [41]:
# pad sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)

Max Sequence Length: 3


In [42]:
sequences

array([[ 2,  1,  3],
       [ 1,  3,  4],
       [ 3,  4,  5],
       [ 4,  5,  6],
       [ 5,  6,  7],
       [ 6,  7,  8],
       [ 7,  8,  9],
       [ 8,  9, 10],
       [ 9, 10, 11],
       [10, 11, 12],
       [11, 12, 13],
       [12, 13,  2],
       [13,  2, 14],
       [ 2, 14, 15],
       [14, 15,  1],
       [15,  1, 16],
       [ 1, 16, 17],
       [16, 17, 18],
       [17, 18,  1],
       [18,  1,  3],
       [ 1,  3, 19],
       [ 3, 19, 20],
       [19, 20, 21]])

In [43]:
# split into input and output elements
sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)

In [44]:
X

array([[ 2,  1],
       [ 1,  3],
       [ 3,  4],
       [ 4,  5],
       [ 5,  6],
       [ 6,  7],
       [ 7,  8],
       [ 8,  9],
       [ 9, 10],
       [10, 11],
       [11, 12],
       [12, 13],
       [13,  2],
       [ 2, 14],
       [14, 15],
       [15,  1],
       [ 1, 16],
       [16, 17],
       [17, 18],
       [18,  1],
       [ 1,  3],
       [ 3, 19],
       [19, 20]])

In [45]:
y

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [46]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_length-1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 2, 10)             220       
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_3 (Dense)              (None, 22)                1122      
Total params: 13,542
Trainable params: 13,542
Non-trainable params: 0
_________________________________________________________________
None


In [47]:
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [48]:
# fit network
model.fit(X, y, epochs=500, verbose=2)

Epoch 1/500
 - 2s - loss: 3.0912 - acc: 0.0435
Epoch 2/500
 - 0s - loss: 3.0904 - acc: 0.0435
Epoch 3/500
 - 0s - loss: 3.0895 - acc: 0.1304
Epoch 4/500
 - 0s - loss: 3.0887 - acc: 0.1304
Epoch 5/500
 - 0s - loss: 3.0878 - acc: 0.1304
Epoch 6/500
 - 0s - loss: 3.0870 - acc: 0.1304
Epoch 7/500
 - 0s - loss: 3.0861 - acc: 0.1304
Epoch 8/500
 - 0s - loss: 3.0852 - acc: 0.1304
Epoch 9/500
 - 0s - loss: 3.0844 - acc: 0.1304
Epoch 10/500
 - 0s - loss: 3.0835 - acc: 0.1304
Epoch 11/500
 - 0s - loss: 3.0825 - acc: 0.1739
Epoch 12/500
 - 0s - loss: 3.0816 - acc: 0.1739
Epoch 13/500
 - 0s - loss: 3.0807 - acc: 0.1739
Epoch 14/500
 - 0s - loss: 3.0797 - acc: 0.1739
Epoch 15/500
 - 0s - loss: 3.0787 - acc: 0.1739
Epoch 16/500
 - 0s - loss: 3.0776 - acc: 0.1739
Epoch 17/500
 - 0s - loss: 3.0766 - acc: 0.1739
Epoch 18/500
 - 0s - loss: 3.0755 - acc: 0.1739
Epoch 19/500
 - 0s - loss: 3.0744 - acc: 0.1739
Epoch 20/500
 - 0s - loss: 3.0732 - acc: 0.1739
Epoch 21/500
 - 0s - loss: 3.0720 - acc: 0.1739
E

Epoch 171/500
 - 0s - loss: 0.8823 - acc: 0.9565
Epoch 172/500
 - 0s - loss: 0.8620 - acc: 0.9565
Epoch 173/500
 - 0s - loss: 0.8419 - acc: 0.9565
Epoch 174/500
 - 0s - loss: 0.8222 - acc: 0.9565
Epoch 175/500
 - 0s - loss: 0.8026 - acc: 0.9565
Epoch 176/500
 - 0s - loss: 0.7833 - acc: 0.9565
Epoch 177/500
 - 0s - loss: 0.7643 - acc: 0.9565
Epoch 178/500
 - 0s - loss: 0.7455 - acc: 0.9565
Epoch 179/500
 - 0s - loss: 0.7270 - acc: 0.9565
Epoch 180/500
 - 0s - loss: 0.7089 - acc: 0.9565
Epoch 181/500
 - 0s - loss: 0.6911 - acc: 0.9565
Epoch 182/500
 - 0s - loss: 0.6736 - acc: 0.9565
Epoch 183/500
 - 0s - loss: 0.6565 - acc: 0.9565
Epoch 184/500
 - 0s - loss: 0.6396 - acc: 0.9565
Epoch 185/500
 - 0s - loss: 0.6230 - acc: 0.9565
Epoch 186/500
 - 0s - loss: 0.6068 - acc: 0.9565
Epoch 187/500
 - 0s - loss: 0.5910 - acc: 0.9565
Epoch 188/500
 - 0s - loss: 0.5756 - acc: 0.9565
Epoch 189/500
 - 0s - loss: 0.5606 - acc: 0.9565
Epoch 190/500
 - 0s - loss: 0.5459 - acc: 0.9565
Epoch 191/500
 - 0s 

Epoch 339/500
 - 0s - loss: 0.0864 - acc: 0.9565
Epoch 340/500
 - 0s - loss: 0.0861 - acc: 0.9565
Epoch 341/500
 - 0s - loss: 0.0859 - acc: 0.9565
Epoch 342/500
 - 0s - loss: 0.0857 - acc: 0.9565
Epoch 343/500
 - 0s - loss: 0.0855 - acc: 0.9565
Epoch 344/500
 - 0s - loss: 0.0852 - acc: 0.9565
Epoch 345/500
 - 0s - loss: 0.0850 - acc: 0.9565
Epoch 346/500
 - 0s - loss: 0.0848 - acc: 0.9565
Epoch 347/500
 - 0s - loss: 0.0846 - acc: 0.9565
Epoch 348/500
 - 0s - loss: 0.0844 - acc: 0.9565
Epoch 349/500
 - 0s - loss: 0.0842 - acc: 0.9565
Epoch 350/500
 - 0s - loss: 0.0840 - acc: 0.9565
Epoch 351/500
 - 0s - loss: 0.0838 - acc: 0.9565
Epoch 352/500
 - 0s - loss: 0.0836 - acc: 0.9565
Epoch 353/500
 - 0s - loss: 0.0834 - acc: 0.9565
Epoch 354/500
 - 0s - loss: 0.0832 - acc: 0.9565
Epoch 355/500
 - 0s - loss: 0.0830 - acc: 0.9565
Epoch 356/500
 - 0s - loss: 0.0828 - acc: 0.9565
Epoch 357/500
 - 0s - loss: 0.0827 - acc: 0.9565
Epoch 358/500
 - 0s - loss: 0.0825 - acc: 0.9565
Epoch 359/500
 - 0s 

In [49]:
# evaluate model
print(generate_seq(model, tokenizer, max_length-1, 'Jack and', 5))
print(generate_seq(model, tokenizer, max_length-1, 'And Jill', 3))
print(generate_seq(model, tokenizer, max_length-1, 'fell down', 5))
print(generate_seq(model, tokenizer, max_length-1, 'pail of', 5))

Jack and jill went up the hill
And Jill went up the
fell down and broke his crown and
pail of water jack fell down and


# https://machinelearningmastery.com/develop-word-based-neural-language-models-python-keras/